# This is for getting my data! YAY!

In [137]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import html

### I have to write a scraper that is able to go through Soundcloud and get the follower and following
List of things that I am able to get:
1. follower list
2. following list
3. tracks posted on profile
4. tracks liked by user

In [138]:
def get_user_info(user_id, list_subject):
    """
    this function gets all the information for the json file that is parsed from requests.
    list_subject tells me whether they want following list or follower list
    I need to also account for the fact that next_href can be NOT None so I need to rerun the fcn until that is true
    """
    # check if next_href is not null, if so, call this function that gets all of the json files before running this function:
    #following_list = []
    url = f'https://api-v2.soundcloud.com/users/{user_id}/{list_subject}?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=200&offset=0&linked_partitioning=1&app_version=1535972763&app_locale=en'

    parsed = requests.get(url).json()
    following_list = []
    
    while parsed.get('next_href') != None: 
        following_list.extend(parsed.get('collection'))
        #print("test") 
        offset = parsed.get('next_href').split('offset=')[1].split('&')[0]
        url = f'https://api-v2.soundcloud.com/users/{user_id}/{list_subject}?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=200&offset={offset}&linked_partitioning=1&app_version=1535972763&app_locale=en'
        parsed = requests.get(url).json()

    following_list.extend(parsed.get('collection'))
    print(f'This user has {len(following_list)} followers')
    return following_list

In [139]:
parsed = get_user_info('58899599', 'followers')
user_list = []

for user in parsed:
    user_dict = {
        'user_id' : user.get('permalink'),
            'city' : user.get('city'),
            'country' : user.get('country_code'),
            'uri' : user.get('uri')
    }
    user_list.append(user_dict)
#print (parsed)

This user has 12 followers


In [140]:
user_df = pd.DataFrame(user_list)
user_df.head(50)

,city,country,uri,user_id
0,Columbus,US,https://api.soundcloud.com/users/281901242,nicasovujep
1,,None,https://api.soundcloud.com/users/253305849,jtlevine
2,,None,https://api.soundcloud.com/users/243602532,rachellawathier44
3,,None,https://api.soundcloud.com/users/243706315,lindseysmason
4,"Berkeley, CA",None,https://api.soundcloud.com/users/165051291,tealbluebear
5,Atlanta,US,https://api.soundcloud.com/users/17821845,spreewellofficial
6,Palo Alto,US,https://api.soundcloud.com/users/169164060,anton-oyung
7,Seoul,KR,https://api.soundcloud.com/users/193228122,soinloveaxing
8,None,None,https://api.soundcloud.com/users/214565468,ashley-tangerine
9,,None,https://api.soundcloud.com/users/119133497,huy-do-32


In [141]:
def get_next_href(user_id, list_subject, offset = '0'):
    """
    This function is for getting the additional list of users after the next_href
    """
    print('entered')
    url = f'https://api-v2.soundcloud.com/users/{user_id}/{list_subject}?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=10&offset={offset}&linked_partitioning=1&app_version=1535972763&app_locale=en'
    parse_json = requests.get(url).json() #starts off being the original json
    print(f'this is the url: {url}')
    
    if parse_json.get('next_href') != None: 
        # if there is a next page, then i want to add it to the existing, and then go to the next page
        #print('recursive')
        #print(full_parsed.get('next_href'))
        offset =parse_json.get('next_href').split('offset=')[1].split('&')[0]
        print ("OffSET is")
        print (offset)
        
        #print(type(full_parsed), type(get_next_href(user_id, list_subject, offset)))
        #if type(get_next_href(user_id, list_subject, offset)) == None:
            #print(get_next_href(user_id, list_subject, offset))
        full_parsed = parse_json.get('collection')
        return full_parsed
        

    else:
        print ('ITS NONE')
        full_parsed = parse_json.get('collection')
        return full_parsed

    


def get_user_info(user_id, list_subject):
    """
    this function gets all the information for the json file that is parsed from requests.
    list_subject tells me whether they want following list or follower list
    I need to also account for the fact that next_href can be NOT None so I need to rerun the fcn until that is true
    """
    # check if next_href is not null, if so, call this function that gets all of the json files before running this function:
    url = f'https://api-v2.soundcloud.com/users/{user_id}/{list_subject}?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=10&offset=0&linked_partitioning=1&app_version=1535972763&app_locale=en'

    
    base_url = "https://api-v2.soundcloud.com/users/58899599/followers"

    params = {
      'client_id': 'STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY',
      'linked_partitioning': 1,
      'limit': 12,
      'offset': 0,
      'app_version': '1535972763',
      'app_locale': 'en'
    }

    parsed = requests.get(base_url, params = params).json()
    #parsed = requests.get(url).json()
    if parsed.get('next_href') != None:
        print(parsed.get('next_href'))
        print(len(parsed.get('collection')))
        full_parsed = get_next_href(user_id, list_subject)
    parsed = parsed.get('collection')
    following_list = []
    for user in parsed:
        user_dict = {
            'user_id' : user.get('permalink'),
            'city' : user.get('city'),
            'country' : user.get('country_code'),
            'uri' : user.get('uri')
            }
        following_list.append(user_dict)
    return following_list


In [142]:
parsed = get_user_info('34359645', 'followers')

In [143]:
len(parsed)

12

# len(parsed)

#### This is for getting followers

In [103]:
client_id = 'YNhF6i7pyrbhTsevCxDfYnjQP7xVyo8h'
user_id = '52154848'
url = f'https://api-v2.soundcloud.com/users/{user_id}/followers?client_id={client_id}&limit=200&offset=0&linked_partitioning=1&app_version=1535640551&app_locale=en'
#url = 'https://bing.com/'
#url = 'https://google.com'
print(requests.get(url).json())
#followers = requests.get(url).json()
#print(followers.keys()) # I need to check if next_href is none, and keep running this until it is



{'collection': [{'avatar_url': 'http://a1.sndcdn.com/images/default_avatar_large.png?1535536648', 'city': None, 'comments_count': 4, 'country_code': None, 'created_at': '2014-11-21T04:10:40Z', 'creator_subscriptions': [{'product': {'id': 'free', 'name': 'Free'}, 'recurring': False, 'hug': False}], 'creator_subscription': {'product': {'id': 'free', 'name': ''}, 'recurring': False, 'hug': False}, 'description': None, 'followers_count': 15, 'followings_count': 181, 'first_name': '', 'full_name': '', 'groups_count': 0, 'id': 123980043, 'kind': 'user', 'last_modified': '2018-05-18T05:59:56Z', 'last_name': '', 'likes_count': 9, 'playlist_likes_count': 3, 'permalink': 'ray_dub_23', 'permalink_url': 'https://soundcloud.com/ray_dub_23', 'playlist_count': 4, 'reposts_count': None, 'track_count': 0, 'uri': 'https://api.soundcloud.com/users/123980043', 'urn': 'soundcloud:users:123980043', 'username': 'ray_dub_23', 'verified': False, 'visuals': None}, {'avatar_url': 'https://i1.sndcdn.com/avatars-0

In [104]:
print('yay')

yay


In [105]:
url = 'https://api-v2.soundcloud.com/users/58899599/followings?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=3&offset=0&linked_partitioning=1&app_version=1535972763&app_locale=en'

parsed = requests.get(url).json()
print(parsed)

{'collection': [{'avatar_url': 'https://i1.sndcdn.com/avatars-000340573119-n8u0an-large.jpg', 'city': 'Los Angeles', 'comments_count': 5, 'country_code': 'US', 'created_at': '2013-03-09T04:22:19Z', 'creator_subscriptions': [{'product': {'id': 'free', 'name': 'Free'}, 'recurring': False, 'hug': False}], 'creator_subscription': {'product': {'id': 'free', 'name': ''}, 'recurring': False, 'hug': False}, 'description': 'Singer/Songwriter/Producer\n\nMGMT:\njesse@9threalmrecords.com', 'followers_count': 218, 'followings_count': 12, 'first_name': '', 'full_name': '', 'groups_count': 0, 'id': 38611053, 'kind': 'user', 'last_modified': '2018-06-01T12:26:49Z', 'last_name': '', 'likes_count': 15, 'playlist_likes_count': 1, 'permalink': 'sarahjakemusic', 'permalink_url': 'https://soundcloud.com/sarahjakemusic', 'playlist_count': 0, 'reposts_count': None, 'track_count': 3, 'uri': 'https://api.soundcloud.com/users/38611053', 'urn': 'soundcloud:users:38611053', 'username': 'Sarah Jake', 'verified': F

In [106]:
len(followers['collection']) # this tells me that I have 12 followers

NameError: name 'followers' is not defined

#### This is for getting a list of people that the user is following.

Something to note, if someone is following more than 200 people (the rate limit), then I have use 'next_href' to find the offset and call the function again.

In [ ]:
url = f'https://api-v2.soundcloud.com/users/58899599/followings?client_id=kLve5Sh7JUSsPl5cCgQ7znRpV2fpzzu8&limit=200&offset=0&linked_partitioning=1&app_version=1535697731&app_locale=en'
following = requests.get(url).json()

print(following)



In [ ]:
following.keys() # i need to check if next_href is None each time because it tells me if there is another page or not

In [ ]:
len(following['collection']) # this tells me that i have 27 followers # this is a list

In [ ]:
following.get('collection')[0]

#### This is for getting a list of songs on the user's profile

domain knowledge, i know a lot of creators will repost their own songs to get more reach... how to account for that?

In [ ]:
url = f'https://api-v2.soundcloud.com/stream/users/58899599?client_id=kLve5Sh7JUSsPl5cCgQ7znRpV2fpzzu8&limit=200&offset=0&linked_partitioning=1&app_version=1535697731&app_locale=en'

streams = requests.get(url).json()
print(streams)



In [ ]:
streams.keys()

In [ ]:
stream_list = [stream.get('track') for stream in streams.get('collection')]
len(stream_list)

#### This is for a list of songs that user liked. Should probably be weighted less than the songs that is posted on the user's profile

In [ ]:
url = f'https://api-v2.soundcloud.com/users/58899599/likes?client_id=kLve5Sh7JUSsPl5cCgQ7znRpV2fpzzu8&limit=200&offset=0&linked_partitioning=1&app_version=1535697731&app_locale=en'

liked = requests.get(url).json()
# print(liked.keys())
# print(liked)


In [ ]:
songs = [song.get('track') for song in liked.get('collection')]
len(songs) #this is now a list of all the songs that the user liked

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# import time

# import os
# chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
# os.environ["webdriver.chrome.driver"] = chromedriver

# driver = webdriver.Chrome(chromedriver)
# driver.get("https://soundcloud.com/jenniperwong/followers")

In [ ]:
# driver = webdriver.Chrome(chromedriver)
# driver.get('https://soundcloud.com/jenniperwong/followers')
# p_element = driver.find_element_by_id(id_='userNetwork')
# print(p_element.text)